# Classification of b-quark jets in the Aleph simulated data

The following is an introduction to using Machine Learning (ML) - in particular Boosted Decision Trees (BDT) - for trying to determine, if an entry in a data file is of one type (signal, ill, guilty, etc.) or another (background, healthy, innocent, etc.).

You may choose between two data samples:
1. A particle physics dataset containing simulated decays of the $Z^0$ boson decaying to a quark and an anti-quark producing two "jets" of particles. The question is, if the jets are from a b-quark (b-jet) or from lighter quarks (l-jet).
3. A "medical" dataset which concers a lifestyle disease in relation to various (transformed) lifestyle variables (reduced in number of variables to match the Aleph b-jet data set).

In the following, we discuss the problem from the b-jet point of view, as this is where the largest size datasets are available. However, we stress that from the point of view of ML, data content (what is being considered) is not essential to know (for now!!!). And knowing the content in details requires domain knowledge, i.e. that you are an expert in the specific field, that the data comes from. This part is very important, but not the focus in this course.

In the end, this exercise is the simple start "outside ML" and moving into the territory of Machine Learning analysis.

### The Data:
The input variables (X) are (used by Aleph for their NN):
* **prob_b**: Probability of being a b-jet from the pointing of the tracks to the vertex.
* **spheri**: Sphericity of the event, i.e. how spherical it is.
* **pt2rel**: The transverse momentum squared of the tracks relative to the jet axis, i.e. width of the jet.
* **multip**: Multiplicity of the jet (in a relative measure).
* **bqvjet**: b-quark vertex of the jet, i.e. the probability of a detached vertex.
* **ptlrel**: Transverse momentum (in GeV) of possible lepton with respect to jet axis (about 0 if no leptons).

Auxilary variables (Z) are (not used by Aleph for their NN):
* energy: Measured energy of the jet in GeV. Should be 45 GeV, but fluctuates.
* cTheta: cos(theta), i.e. the polar angle of the jet with respect to the beam axis. Note, that the detector works best in the central region (|cTheta| small) and less well in the forward regions.
* phi:    The azimuth angle of the jet. As the Aleph detector was essentially uniform in phi, this should not matter (much).

The target variable (Y) is:
* **isb**:    1 if it is from a b-quark and 0, if it is not.

Finally, those before you (the Aleph collaboration in the mid 90'ies) produced a Neural Net (6 input variables, two hidden layers with 10 neurons in each, and 1 output varible) based classification variable, which you can compare to (and compete with?):
* **nnbjet**: Value of original Aleph b-jet tagging algorithm, using only the last six variables (for reference).

In case you choose **the medical data**, the variables to use as input (X) are: **Qsocial, BMI, Roccupat, Rgenetic, Rdietary, and Rhormonn** (reflecting Quantiles and Ratios of medical measurements). The target variable (Y) is (naturally): **TrulyIll**, and you can compare your results to the average of doctors: **DocScore**.


### The Task:
Thus, the task before you is to produce functions (non-ML and ML algorithm), which given the input variables X provides an output variable estimate, Y_est, which is "closest possible" to the target variable, Y. The "closest possible" is left to the user to define in a _Loss Function_, which we will discuss further later. In classification problems (such as this), the typical loss function to use is ["Cross Entropy"](https://en.wikipedia.org/wiki/Cross_entropy).

You should approach the problem in two ways:
* Initially (i.e. first 15-30 minutes), simply with "if"-statements making requirements on certain variables. This corresponds to selecting "boxes" in the input variable space (typically called "X"). One could also try a linear combination of input variables (i.e. a Fisher discriminant), which corresponds to a plane in the X-space. Such a solution is fast and transparent (very good), but as the data contains non-linear correlations, it is likely to approximate.<br>
**The point of this step is NOT to make a great algorith, but rather to set up comparing different algorithms!**

* Next using Machine Learning (ML) methods. We will during the first week try both Boosted Decision Tree (BDT) based and Neural Net (NN) based methods, and see how complicated (or not) it is to get a good solution, and how much better it performs compared to the "classic" selection method.

Once you obtain a classification of b-jets vs. non-b-jets, think about how to quantify the quality of your algorithm. Once you have a "metric" for doing so, try to compare it to the NN result of the Aleph collaboration, given by the variable "nnbjet". It is based on a neural network with 6 input variables (prob_b, spheri, pt2rel, multip, bqvjet, and ptlrel), and two hidden layers each with 10 nodes in. Can you do better?

Don't hold back in drawing inspiration from "ML_MethodsDemos.ipynb" (in Week0) or the vast internet. BDT suggestions might be [XGBoost](https://xgboost.readthedocs.io/en/release_3.0.0/) or [LightGBM](https://lightgbm.readthedocs.io/en/stable/). NN suggestions might be [PyTorch](https://pytorch.org/) or [TensorFlow](https://www.tensorflow.org/).

* Author: Troels C. Petersen (NBI)
* Email:  petersen@nbi.dk
* Date:   15th of April 2025

In [ ]:
from __future__ import print_function, division   # Ensures Python3 printing & division standard
import pandas as pd 
from pandas import Series, DataFrame 
from matplotlib import pyplot as plt
import numpy as np

SavePlots = False

## Import and inspect the data:

In [ ]:
# Read the data and print the variables:
data = pd.DataFrame(np.genfromtxt('AlephBtag_MC_train_Nev5000.csv', names=True))
# data = pd.DataFrame(np.genfromtxt('Medical_Npatients5000.csv', names=True))

variables = data.columns
print(variables.values)

# Decide on which variables to use for input (X) and what defines the label (Y):
input_variables = variables[(variables != 'nnbjet') & (variables != 'isb') & (variables != 'energy') & (variables != 'cTheta') & (variables != 'phi')]
input_data      = data[input_variables]
truth_data      = data['isb']
benchmark_data  = data['nnbjet']
print("  Variables used for training: ", input_variables.values)

In [ ]:
# Look at the distribution of the input variables for each event type:
mask = (data['isb'] == 1)      # Mask to divide the event types

fig, ax = plt.subplots(figsize=(12, 6))
hist_prob_b_bjets = ax.hist(data['prob_b'][mask],  bins=100, range=(0.0, 1.0), histtype='step', linewidth=2, label='prob_b_bjets', color='blue')
hist_prob_b_ljets = ax.hist(data['prob_b'][~mask], bins=100, range=(0.0, 1.0), histtype='step', linewidth=2, label='prob_b_ljets', color='red')
ax.set_xlabel("Probability of b-quark based on track impact parameters")
ax.set_ylabel("Frequency / 0.01")
ax.set_title("Distribution of prob_b")
ax.set_yscale("log")
ax.legend(loc='best')
ax.grid(axis='y')
fig.tight_layout()

if SavePlots :
    fig.savefig('Hist_prob_b.pdf', dpi=600)

## Selection:

Imagine that you had never heard of Machine Learning (ML), and had to select b-jets without! This would probably boil down to doing so simply based on "if"-sentences, as shown below: 

In [ ]:
# Give the variables logic names, and define them early, so that they only need to be changed
# in ONE place (also to ensures consistency!):
cut_prop_b  = 0.15

# If prob_b indicate b-quark, call it a b-quark, otherwise not! You can expand on this yourself at will.
bquark=[]
for i in np.arange(len(data['prob_b'])):
    if (data['prob_b'][i] > cut_prop_b) : bquark.append(1)
    else : bquark.append(0)

## Evaluate the selection:

In [ ]:
def evaluate(bquark) :
    N = [[0,0], [0,0]]   # Make a list of lists (i.e. confusion matrix) for counting successes/failures.
    for i in np.arange(len(data['isb'])):
        if (bquark[i] == 0 and data['isb'][i] == 0) : N[0][0] += 1
        if (bquark[i] == 0 and data['isb'][i] == 1) : N[0][1] += 1
        if (bquark[i] == 1 and data['isb'][i] == 0) : N[1][0] += 1
        if (bquark[i] == 1 and data['isb'][i] == 1) : N[1][1] += 1
    fracWrong = float(N[0][1]+N[1][0])/float(len(data['isb']))
    accuracy = 1.0 - fracWrong
    return N, accuracy, fracWrong

In [ ]:
N, accuracy, fracWrong = evaluate(bquark)
print("\nRESULT OF HUMAN ATTEMPT AT A SELECTION:")
print("  First number in parenthesis is the estimate, second is the MC truth (i.e. label):")
print("  True-Negative (0,0)  = ", N[0][0])
print("  False-Negative (0,1) = ", N[0][1])
print("  False-Positive (1,0) = ", N[1][0])
print("  True-Positive (1,1)  = ", N[1][1])
print("    Fraction wrong            = ( (0,1) + (1,0) ) / sum = ", fracWrong)
print("    Fraction right (accuracy) = ( (0,0) + (1,1) ) / sum = ", accuracy)

In [ ]:
# Compare with Aleph NN-approach from 1990'ies (choosing a reasonable selection cut on "nnbjet"):
bquark=[]
for i in np.arange(len(data['nnbjet'])):
    if   (data['nnbjet'][i] > 0.82) : bquark.append(1)
    else : bquark.append(0)

N, accuracy, fracWrong = evaluate(bquark)
print("\nALEPH BJET TAG:")
print("  First number in parenthesis is the estimate, second is the MC truth (i.e. label):")
print("  True-Negative (0,0)  = ", N[0][0])
print("  False-Negative (0,1) = ", N[0][1])
print("  False-Positive (1,0) = ", N[1][0])
print("  True-Positive (1,1)  = ", N[1][1])
print("    Fraction wrong            = ( (0,1) + (1,0) ) / sum = ", fracWrong)
print("    Fraction right (accuracy) = ( (0,0) + (1,1) ) / sum = ", accuracy)

## Questions:

1. Start by plotting the six "Aleph classification variables" and calculate their correlation matrices for signal and background, and see which seems to separate and correlate most (optional).

2. Above, the scoring is simply done by considering the fraction of wrong estimates. Think about what the alternatives could be, especially if you were to give a continuous score like the Aleph-NN. This scoring is called the "loss function".

3. <b>Draw ROC curves for each of these six separately, to quantify the separation. Also draw the Aleph-NN variable (nnbjet).</b> Is the Aleph-NN better? Are any of the variables "seemingly worthless"? (we will return to this point later).<br>
Note that decision variables (i.e. 0 and 1) yield a single point in the ROC curve plot.

5. <b>Try to get a BDT to make predictions based on the six given input variables. Plot the result in the ROC curve, and see if you managed to compete or even improve upon the Aleph-NN scores.</b>

6. <b>Think about what data you based your predictions on? Was it the same data that you trained the algorithm on? And if that is the case, is that reasonable?<b>

7. Does including more data (50000 instead of 5000 events) improve your performance?

8. Does including the kinematic variables 'energy', 'cTheta', and 'phi' (which are not related to jet type) improve your performance?

## Learning points:

From this exercise you should:

1. get a feel for the problem at hand, namely to separate two populations in a 6-dimensional space. It is hard to imagine, yet with simple cuts you should be able to get "some performance", though never close to the Aleph-NN.

2. learn, that it is hard to do "by hand", but that at least the fact that you have known cases (i.e. labels) makes you capable of getting some performance.

3. be able to draw ROC curves to compare performances between different variables and scores (floating or discrete).

4. know that you can (typically) improve the performance through the use of Machine Learning (ML).

5. <b>be capable not only of getting ML results, but also confident in optimising them, and certainly proficient in interpreting them.</b>

---

# Classify B-jets using MLPclassifier:

The following is a solution example based on MLPclassifier, which is know to be simple and included in the SciKit Learn package.

## Notes about Neural Network based ML solution

The following is a solution based on MLPclassifier (MLP = MultiLayer Perceptron, essentially a (feed forward) Neural Network). Examples of different MLPclassifier solutions can be found here:
* https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
* https://michael-fuchs-python.netlify.app/2021/02/03/nn-multi-layer-perceptron-classifier-mlpclassifier/

Similar solutions can be produced with [PyTorch](https://pytorch.org/) or [TensorFlow](https://www.tensorflow.org/)

Generally, the steps are:
1. Define training, validation, and testing samples.
2. Normalise the input (and possibly output) variables to a mean and variance around unity (i.e. 0 and 1).
3. Define the feature (input) and target (output) variables.
4. Define the loss function (i.e. what it should aim to optimize).
5. Fit for a model.
6. Apply the model to the test set, and determine performance.

Usually one would apply further checks/regularization/standardization of data before training a model, but this data has already been very nicely "prepared". It won't stay that way!

## MLPclassifier hyperparameters:

There are several settings that one can run MLPclassifier with, see this [list of hyperparameters for MLPclassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import time

start=time.time()

# Split data set into training and test set. We choose a 75:25 division here. 

# Dataset is shuffeled before the split (to avoid any ordering). By using a fixed
# random seed number (42), we can rerun and obtain the same result (for reproducibility!).
input_train, input_test, truth_train, truth_test, benchmark_train, benchmark_test = \
    train_test_split(input_data, truth_data, benchmark_data, test_size=0.25, random_state=42)

clf = MLPClassifier(max_iter=20000,
                    n_iter_no_change=100,           # Number of iterations without improvement before stopping
                    solver='adam',                  # Standard for minimising
                    activation='logistic',          # Standard function (but slightly slower)
                    hidden_layer_sizes=(10, 10),    # Size of network (same as ALEPH)
                    learning_rate='invscaling',     # Other options are "constant" and "adaptive"
                    random_state=42)

# Train the model:
clf.fit(input_train, truth_train)

# Make predictions (NOTE: The output is two columns: p_signal and p_background = 1 - p_signal):
y_score_MLP = clf.predict_proba(input_test)

# Print the time usage:
end = time.time()
print(f"Time used by MLPClassifier: {(end-start)*1000:.1f} ms")

In [ ]:
# Evaluate:
fpr, tpr, _ = roc_curve(truth_test, y_score_MLP[:,1])              # False/True Positive Rate for our model
fpr_nnbjet, tpr_nnbjet, _ = roc_curve(truth_test, benchmark_test)  # False/True Positive Rate for Aleph NNbjet

# We can now calculate the Area-Under-the-Curve (AUC) scores of these ROC-curves:
auc_score = auc(fpr,tpr)                        # This is the AUC score for our model
auc_score_nnbjet = auc(fpr_nnbjet, tpr_nnbjet)  # This is the AUC score for Aleph NNbjet

# Let's plot the ROC curves for these results:
fig = plt.figure(figsize = [10,10])
plt.title('Model Comparison (ROC curves)', size = 16)
plt.plot(fpr, tpr, label=f'Our MLPclassifier model (AUC = {auc_score:5.3f})')
plt.plot(fpr_nnbjet, tpr_nnbjet, label = f'Aleph NNbjet (AUC = {auc_score_nnbjet:5.3f})')
plt.legend(fontsize=16)
plt.xlabel('False Postive Rate', size=16)
plt.ylabel('True Positive Rate', size=16)
plt.show()

## Notes about solution:

The above solution took a few second to run, works for any number of input variables, and produces a result very close to the optimal. You _also_ gotta love this one, even if it is slower than the BDT, requires more variable preparation (only nicely behaved values without any NaNs), and takes more "care and feeding" to converge. The strength is the versatility of the NN approach, which we will explore in the weeks to come.

Why doesn't it reach (or surpass) the Aleph-NN solution? Well, we have not optimised the performance of the MLPclassifier algorithm (i.e. optimized the hyper parameters - more to come on that), nor trained it on large amounts of data. Try to do that yourself.

# Classify B-jets using TensorFlow:

This is a solution example using TensorFlow (NN based).

The example is built with inspiration from
https://blog.cmgresearch.com/2020/09/06/tensorflow-binary-classification.html

The link contains additional explanitory text and short 5-minute youtube video explaining core concepts.

__Note:
The solution NNs below (both in TensorFlow and PyTorch) are very simple and un-optimized models. Yet they manage to  achieves a slightly bit higher AUC score than nnbjet (higher is better). But they can be further optimised, which we will be working on more, in particular for the initial/individual project.__

# Read the data

...and choose input and target variables:

A few worlds on the variables and feature choices:
* 'isb' is our binary truth variables. If isb = 1 then it's a b-quark and isb = 0 if it is not.
   Because this is our truth, we must not include it as the input to our model.
* 'nnbjet' is our "competitor" e.g. a model we are supposed to benchmark against.
   Therefore 'nnbjet' shouldn't be in our input either.
* 'energy', 'cTheta', and 'phi' are kinematic variables of the jet, and not about the jet type.
   Though they might help, they were not in the original Aleph NN (nnbjet), and to compare, we omit them.

Usually one would apply further checks/regularization/standardization of data at this step, but this data has already been "prepared", so we'll move onto seperate the data into input, truth and benchmark:

In [ ]:
# Load 2nd version of the Aleph Data.
# path = 'AlephBtag_MC_train_Nev5000.csv'
path = 'AlephBtag_MC_train_Nev50000.csv'
data = pd.DataFrame(np.genfromtxt(path, names=True))
variables = data.columns
print(variables)

input_variables = variables[(variables != 'nnbjet') & (variables != 'isb') & (variables != 'energy') & (variables != 'cTheta') & (variables != 'phi')] #Variables not used in Aleph
# input_variables = variables[(variables != 'nnbjet') & (variables != 'isb')]          #Variables not to be used at all
input_data      = data[input_variables]
truth_data      = data['isb']
benchmark_data  = data['nnbjet']
print(input_variables)

In [ ]:
input_data.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Always split your dataset into train and validation part. We choose a 75:25 division here. Shuffels dataset before split. By using a number (42), it creates a random seed so you can rerun and obtain the same result.
input_train, input_valid, truth_train, truth_valid = train_test_split(input_data, truth_data, test_size=0.25, random_state=42)
 
# Create a NN. Loss function is BinaryCrossEntropy. Output layer has 1 node;the prediction for isb. Learning rate defaults to 0.001. 
model = Sequential([
    Dense(9,activation='relu',name='input_layer1'),
    Dense(24,activation='relu',name='hidden_layer1'),
    Dense(12,activation='relu',name='hidden_layer2'),
    Dense(1, activation='sigmoid', name='output')])
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.losses.BinaryCrossentropy()])

print('--------- TRAINING ---------')
history = model.fit(x = np.array(input_train), y = np.array(truth_train), validation_data=(np.array(input_valid), np.array(truth_valid)), epochs = 7)  
## This trains the model on input_train by comparing to the true values in truth_train. After every epoch of training, the model is evaluated on the validation dataset, 
## namely input_valid and truth_valid.

In [ ]:
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

fig =  plt.figure()
plt.plot(training_loss,label = 'training loss')
plt.plot(training_loss,'o')
plt.plot(validation_loss, label = 'validation loss')
plt.plot(validation_loss, 'o')
plt.legend()
plt.xticks(size = 12)
plt.yticks(size = 12)
plt.show()

#### Discussion:
As you can see, after 1st epoch the validation loss and training loss match/cross each other. This is important! Do you know why?

Now we have a trained model and we're ready to make predictions. Usually, one would have a test set (so in total one would have; a training set, a validation set AND a test set). But for simplicity, let's just predict on the validation sample. This is OK because the model has not trained on this set - if we asked the model to predict on examples on which it has trained, we would be cheating!

In [ ]:
# Make predictions on input_valid and plot performance (ROC curve). Notice we're not giving it any truth values!
predictions = model.predict(input_valid)

fpr, tpr, _ = roc_curve(truth_valid, predictions)      # False Positive Rate and True Positive Rate for our model
fpr_nnbjet, tpr_nnbjet, _ = roc_curve(truth_data, benchmark_data) # False Positive Rate and True Positive Rate for ALEPH model

# We can now calculate the AUC scores of these ROC-curves:
auc_score = auc(fpr,tpr)                         # this is auc score for our model
auc_score_nnbjet = auc(fpr_nnbjet, tpr_nnbjet)   # this is the auc score for nnbjet

# Plot the results:
fig = plt.figure(figsize = [10,10])
plt.title('ROC Comparison', size = 18)
plt.xlabel('False Postive Rate', size = 18)
plt.ylabel('True Positive Rate', size = 18)
plt.plot(fpr,        tpr,        label = f'Our model (AUC = {auc_score:6.4f})')
plt.plot(fpr_nnbjet, tpr_nnbjet, label = f'nnbjet (AUC = {auc_score_nnbjet:6.4f}')
plt.legend(loc='lower right', fontsize=18)
plt.show()

# Classify B-jets using PyTorch:

This is a solution example using PyTorch (NN based).

The example is built with inspiration from
https://towardsdatascience.com/pytorch-tabular-binary-classification-a0368da5bb89

The link contains additional explanitory text and short 5-minute youtube video explaining core concepts.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from scipy.special import expit

#### PyTorch "specials":
PyTorch requires that we put this data into the pytorch Dataset class, such that we can extract it during training.

PyTorch generally wants things to be written into functions and classes, which makes it slightly less easy to "just use", but once you get acquainted with this, PyTorch is a powerful and versatile tool.

In [ ]:
# Creating a Pythorch dataset class:
class MyDataset(Dataset):    
    def __init__(self, X_data, y_data):
        self.input = X_data
        self.truth = y_data
        
    def __getitem__(self, index):
        return self.input[index], self.truth[index]
        
    def __len__ (self):
        return len(self.input)

# In pytorch, there is an additional step of turning your data into tensors
train_data = MyDataset(torch.FloatTensor(np.array(input_train)), 
                       torch.FloatTensor(np.array(truth_train)))
valid_data = MyDataset(torch.FloatTensor(np.array(input_valid)), 
                       torch.FloatTensor(np.array(truth_valid)))

# We can now access input_train via train_data.input and truth_train via train_data.truth,
# and similarly for input_valid and truth_valid:
print(train_data.input)
print(train_data.truth)

In [ ]:
# Define the model:
class OurModel(nn.Module):
    def __init__(self):
        super(OurModel, self).__init__()        # Here we define the layers.
        self.input_layer = nn.Linear(6, 24)     # In pytorch, you define the input and output edges (i.e. layer sizes).
        self.hidden_layer1 = nn.Linear(24, 24)
        self.hidden_layer2 = nn.Linear(24, 12)
        self.output_layer = nn.Linear(12, 2) 
        self.relu = nn.ReLU()
        
    def forward(self, inputs):                  # Here we define how data passes through the layers. 
        x = self.input_layer(inputs)            # Also here, pytorch is a bit more explicit in defining the layers and activation function separately
        x = self.relu(x)
        x = self.hidden_layer1(x)
        x = self.relu(x)
        x = self.hidden_layer2(x)
        x = self.relu(x)
        x = self.output_layer(x)
        return x

In [ ]:
# Defining training loop, validation, and prediction:
def Train(model, optimizer, loss_function, train_loader, validation_loader, device, epochs):
    validation_loss = []
    training_loss   = []
    model.train()
    for e in range(0, epochs):
        epoch_loss = 0
        n_minibatches = 0
        for input_train_batch, truth_train_batch in train_loader:
            input_train_batch, truth_train_batch = input_train_batch.to(device), truth_train_batch.to(device)
            optimizer.zero_grad()
            prediction = model(input_train_batch)  # This asks our model to produce predictions on the training batch            
            loss = loss_function(prediction, truth_train_batch.long())  # This calculates the loss
            loss.backward()                                             # This initiates the backpropagation
            optimizer.step()
            epoch_loss += loss.item()
            n_minibatches += 1
        
        # Now that the model have trained 1 epoch, we evaluate the model on the validation set and print the progress:
        valid_loss = Validate(model, validation_loader, device, loss_function)
        validation_loss.append(valid_loss)
        training_loss.append(epoch_loss/n_minibatches)
        print('EPOCH: %s | training loss: %s  | validation loss: %s'%(e+1,round(epoch_loss/n_minibatches,3), round(valid_loss, 3)))
    return training_loss, validation_loss


def Validate(model, validation_loader, device, loss_function):
    model.eval()
    n_batches  = 0
    validation_loss = 0
    with torch.no_grad():
        for input_valid_batch, truth_valid_batch in validation_loader:
            input_valid_batch, truth_valid_batch = input_valid_batch.to(device), truth_valid_batch.to(device)
            prediction = model(input_valid_batch)
            loss = loss_function(prediction, truth_valid_batch.long())
            validation_loss += loss.item()
            n_batches += 1
    validation_loss = validation_loss/n_batches
    return validation_loss


def Predict(model, prediction_loader, device):
    model.eval()
    predictions = []
    print('PREDICTING!')
    with torch.no_grad():
        for input_pred_batch, _ in validation_loader:
            input_pred_batch = input_pred_batch.to(device)
            prediction = model(input_pred_batch)
            predictions.extend(prediction.numpy())
    print('Done Predicting!')
    return predictions

In [ ]:
# Now everything is ready, and we thus define the optimisation (hyper-) parameters:
learning_rate = 1e-3      # The step size in the direction of "good" from stocastic gradient descent (important!)
batch_size    = 32        # The size of the batches used for each of the stocastic gradient descent calculations
n_epochs      = 8         # Number of epochs, i.e. times that we run through the entire dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = OurModel() 
model.to(device)          # Mount the model to the selected device. Either CPU or GPU.
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = CrossEntropyLoss()
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=valid_data, batch_size=batch_size)

training_loss, validation_loss = Train(model, optimizer, loss_function, train_loader, validation_loader, device, n_epochs)

In [ ]:
fig =  plt.figure()
plt.plot(training_loss,label = 'training loss')
plt.plot(training_loss,'o')
plt.plot(validation_loss, label = 'validation loss')
plt.plot(validation_loss, 'o')
plt.legend()
plt.xticks(size = 12)
plt.yticks(size = 12)
plt.show()

In [ ]:
predictions = Predict(model, validation_loader, device) # Make predictions on input_valid. Notice we're not giving it any truth values!
# print("\nRaw predictions: \n", predictions[:10])

# The output of our model is raw "logits" from the final output layer.
# This means it produces a pseudo score for each class (a score for 0 and a score for 1). 
# The function "expit" converts this logit to a number in [0,1].
# We then combine the logit scores such that our_score = (1_score) / (1_score + 0_score).
predictions = pd.DataFrame(predictions)
predictions.columns = ['not_bquark', 'bquark']
predictions['not_bquark'] = expit(predictions['not_bquark'])
predictions['bquark'] = expit(predictions['bquark'])
predictions = predictions['bquark']/(predictions['bquark'] + predictions['not_bquark'])

#print("\nTransformed predictions: \n", predictions[:10])

In [ ]:
fpr, tpr, _ = roc_curve(truth_valid, predictions)      # False Positive Rate and True Positive Rate for our model
fpr_nnbjet, tpr_nnbjet, _ = roc_curve(truth_data, benchmark_data) # False Positive Rate and True Positive Rate for ALEPH model

# We can now calculate the AUC scores of these ROC-curves:
auc_score = auc(fpr,tpr)                         # this is auc score for our model
auc_score_nnbjet = auc(fpr_nnbjet, tpr_nnbjet)   # this is the auc score for nnbjet

# Plot the results:
fig = plt.figure(figsize = [10,10])
plt.title('ROC Comparison', size = 18)
plt.xlabel('False Postive Rate', size = 18)
plt.ylabel('True Positive Rate', size = 18)
plt.plot(fpr,        tpr,        label = f'Our model (AUC = {auc_score:6.4f})')
plt.plot(fpr_nnbjet, tpr_nnbjet, label = f'nnbjet (AUC = {auc_score_nnbjet:6.4f}')
plt.legend(loc='lower right', fontsize=18)
plt.show()